# 🚀 YOLOv12-Face Enhanced Training

Notebook d'entraînement du modèle YOLOv12-Face avec modules d'attention Enhanced.

## 📦 Installation des Dépendances

In [ ]:
# Installer les dépendances nécessaires
!pip install ultralytics gdown opencv-python -q

# Vérifier l'installation
import torch
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

print("📦 Dépendances installées avec succès !")

## 🔧 Configuration et Restauration

In [ ]:
import os
import shutil
from pathlib import Path

# Configuration des chemins (compatible environnements cloud)
try:
    PROJECT_ROOT = Path.cwd()
except (FileNotFoundError, OSError):
    # Fallback pour environnements cloud
    PROJECT_ROOT = Path('/content') if Path('/content').exists() else Path.home()
    print(f"⚠️ Utilisation du répertoire: {PROJECT_ROOT}")

SCRIPTS_DIR = PROJECT_ROOT / "scripts"
CONFIGS_DIR = SCRIPTS_DIR / "configs"

print(f"📁 Projet: {PROJECT_ROOT}")
print(f"📁 Scripts: {SCRIPTS_DIR}")
print(f"📁 Configs: {CONFIGS_DIR}")

# Créer structure de sauvegarde si nécessaire
SCRIPTS_DIR.mkdir(exist_ok=True)
CONFIGS_DIR.mkdir(exist_ok=True)
(CONFIGS_DIR / 'datasets').mkdir(exist_ok=True)
(CONFIGS_DIR / 'models' / 'v12').mkdir(parents=True, exist_ok=True)
(CONFIGS_DIR / 'modules').mkdir(exist_ok=True)

print("✅ Structure créée")

In [ ]:
def restore_configs():
    """Restaure nos fichiers personnalisés"""
    files_map = [
        ('datasets/widerface.yaml', 'ultralytics/cfg/datasets/widerface.yaml'),
        ('models/v12/yolov12-face.yaml', 'ultralytics/cfg/models/v12/yolov12-face.yaml'),
        ('models/v12/yolov12-face-enhanced.yaml', 'ultralytics/cfg/models/v12/yolov12-face-enhanced.yaml'),
        ('modules/enhanced.py', 'ultralytics/nn/modules/enhanced.py')
    ]
    
    restored_count = 0
    for src, dst in files_map:
        src_path = CONFIGS_DIR / src
        dst_path = PROJECT_ROOT / dst
        
        if src_path.exists():
            dst_path.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src_path, dst_path)
            print(f"🔄 Restauré: {dst}")
            restored_count += 1
        else:
            print(f"⚠️ Fichier manquant: {src}")
    
    # Mettre à jour __init__.py
    init_file = PROJECT_ROOT / 'ultralytics/nn/modules/__init__.py'
    if init_file.exists():
        with open(init_file, 'r') as f:
            content = f.read()
        
        if 'from .enhanced import *' not in content:
            lines = content.split('\n')
            for i, line in enumerate(lines):
                if line.startswith('__all__'):
                    lines.insert(i, 'from .enhanced import *')
                    break
            
            with open(init_file, 'w') as f:
                f.write('\n'.join(lines))
            print("✅ __init__.py mis à jour")
    
    return restored_count

# Restaurer les configurations
restored = restore_configs()
print(f"\n📊 {restored} fichiers restaurés")

## ✅ Vérification

In [ ]:
# Import d'Ultralytics
from ultralytics import YOLO
print("✅ Ultralytics importé")

# Test modules enhanced
try:
    from ultralytics.nn.modules.enhanced import A2Module, RELAN
    print("✅ Modules Enhanced importés")
    
    # Test rapide
    x = torch.randn(1, 64, 32, 32)
    a2 = A2Module(64, 64)
    out = a2(x)
    print(f"✅ Test A2Module: {x.shape} -> {out.shape}")
    
except ImportError as e:
    print(f"❌ Modules Enhanced non trouvés: {e}")
    print("💡 Vérifiez que la restauration des fichiers a réussi")
except Exception as e:
    print(f"❌ Erreur de test: {e}")

## 🏋️ Entraînement Enhanced

In [ ]:
# Configuration
MODEL_CONFIG = "ultralytics/cfg/models/v12/yolov12-face-enhanced.yaml"
DATA_CONFIG = "ultralytics/cfg/datasets/widerface.yaml"
EPOCHS = 100
BATCH_SIZE = 16

print(f"📊 Configuration:")
print(f"   Model: {MODEL_CONFIG}")
print(f"   Data: {DATA_CONFIG}")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch: {BATCH_SIZE}")

# Créer modèle
model = YOLO(MODEL_CONFIG)
print("✅ Modèle Enhanced créé")

In [ ]:
# Entraînement
print("🚀 Démarrage de l'entraînement...")

results = model.train(
    data=DATA_CONFIG,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=640,
    device=0 if torch.cuda.is_available() else 'cpu',
    project='runs/detect',
    name='yolov12-face-enhanced',
    save=True,
    plots=True
)

print("🎉 Entraînement terminé !")

## 📊 Résultats

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Charger résultats
results_csv = Path('runs/detect/yolov12-face-enhanced/results.csv')

if results_csv.exists():
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()
    
    # Graphique simple
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    if 'metrics/mAP50(B)' in df.columns:
        plt.plot(df['epoch'], df['metrics/mAP50(B)'])
        plt.title('mAP@0.5')
        plt.grid(True)
    
    plt.subplot(2, 2, 2)
    if 'metrics/precision(B)' in df.columns:
        plt.plot(df['epoch'], df['metrics/precision(B)'])
        plt.title('Precision')
        plt.grid(True)
    
    plt.subplot(2, 2, 3)
    if 'metrics/recall(B)' in df.columns:
        plt.plot(df['epoch'], df['metrics/recall(B)'])
        plt.title('Recall')
        plt.grid(True)
    
    plt.subplot(2, 2, 4)
    if 'train/box_loss' in df.columns:
        plt.plot(df['epoch'], df['train/box_loss'])
        plt.title('Loss')
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Meilleures métriques
    print("📈 Meilleures performances:")
    if 'metrics/mAP50(B)' in df.columns:
        print(f"   mAP@0.5: {df['metrics/mAP50(B)'].max():.4f}")
    if 'metrics/precision(B)' in df.columns:
        print(f"   Precision: {df['metrics/precision(B)'].max():.4f}")
    if 'metrics/recall(B)' in df.columns:
        print(f"   Recall: {df['metrics/recall(B)'].max():.4f}")

else:
    print("❌ Résultats non trouvés")

## 🧪 Test du Modèle

In [ ]:
# Charger le meilleur modèle
best_model = Path('runs/detect/yolov12-face-enhanced/weights/best.pt')

if best_model.exists():
    model_trained = YOLO(str(best_model))
    print(f"📁 Modèle chargé: {best_model}")
    
    # Validation
    val_results = model_trained.val(data=DATA_CONFIG)
    
    print("📊 Résultats finaux:")
    print(f"   mAP@0.5: {val_results.box.map50:.4f}")
    print(f"   mAP@0.5-0.95: {val_results.box.map:.4f}")
    print(f"   Precision: {val_results.box.mp:.4f}")
    print(f"   Recall: {val_results.box.mr:.4f}")
    
    # Export ONNX
    try:
        onnx_path = model_trained.export(format='onnx', imgsz=640)
        print(f"✅ Modèle exporté: {onnx_path}")
    except Exception as e:
        print(f"⚠️ Export échoué: {e}")

else:
    print("❌ Modèle non trouvé")

## 🎉 Terminé !

### ✅ Ce qui a été fait :
- Installation automatique des dépendances
- Restauration des configurations Enhanced
- Entraînement du modèle YOLOv12-Face Enhanced
- Analyse des performances
- Export du modèle

### 📁 Fichiers générés :
- `runs/detect/yolov12-face-enhanced/weights/best.pt`
- `runs/detect/yolov12-face-enhanced/results.csv`
- Modèle ONNX exporté

**Le modèle Enhanced est prêt pour la production ! 🚀**